# 🍬 Candy Distributor - Exploratory Data Analysis

This notebook performs EDA on the Candy Distributor sales data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Data path
DATA_PATH = Path('../DataSource')

## 1. Load Data

In [ ]:
# Load all datasets
sales = pd.read_csv(DATA_PATH / 'Candy_Sales.csv')
products = pd.read_csv(DATA_PATH / 'Candy_Products.csv')
factories = pd.read_csv(DATA_PATH / 'Candy_Factories.csv')
targets = pd.read_csv(DATA_PATH / 'Candy_Targets.csv')
geo = pd.read_csv(DATA_PATH / 'uszips.csv')

print(f"Sales: {sales.shape}")
print(f"Products: {products.shape}")
print(f"Factories: {factories.shape}")
print(f"Targets: {targets.shape}")
print(f"Geography: {geo.shape}")

In [ ]:
# Preview sales data
sales.head()

In [ ]:
# Data types and info
sales.info()

## 2. Data Cleaning

In [ ]:
# Convert date columns
sales['Order Date'] = pd.to_datetime(sales['Order Date'])
sales['Ship Date'] = pd.to_datetime(sales['Ship Date'])

# Extract date components
sales['Year'] = sales['Order Date'].dt.year
sales['Month'] = sales['Order Date'].dt.month
sales['Quarter'] = sales['Order Date'].dt.quarter
sales['DayOfWeek'] = sales['Order Date'].dt.dayofweek
sales['YearMonth'] = sales['Order Date'].dt.to_period('M').astype(str)

# Calculate shipping days
sales['Ship Days'] = (sales['Ship Date'] - sales['Order Date']).dt.days

print("Date range:", sales['Order Date'].min(), "to", sales['Order Date'].max())

In [ ]:
# Check for missing values
sales.isnull().sum()

## 3. Summary Statistics

In [ ]:
# Key metrics
print("=" * 50)
print("KEY BUSINESS METRICS")
print("=" * 50)
print(f"Total Sales: ${sales['Sales'].sum():,.2f}")
print(f"Total Cost: ${sales['Cost'].sum():,.2f}")
print(f"Gross Profit: ${sales['Gross Profit'].sum():,.2f}")
print(f"Gross Margin: {sales['Gross Profit'].sum() / sales['Sales'].sum() * 100:.1f}%")
print(f"Total Units: {sales['Units'].sum():,}")
print(f"Total Orders: {sales['Order ID'].nunique():,}")
print(f"Unique Customers: {sales['Customer ID'].nunique():,}")
print(f"Avg Order Value: ${sales.groupby('Order ID')['Sales'].sum().mean():.2f}")

In [ ]:
# Descriptive statistics
sales[['Sales', 'Units', 'Cost', 'Gross Profit']].describe()

## 4. Time Series Analysis

In [ ]:
# Monthly sales trend
monthly = sales.groupby('YearMonth').agg({
    'Sales': 'sum',
    'Units': 'sum',
    'Order ID': 'nunique'
}).reset_index()
monthly.columns = ['YearMonth', 'Sales', 'Units', 'Orders']

fig = px.line(monthly, x='YearMonth', y='Sales', 
              title='Monthly Sales Trend',
              labels={'Sales': 'Sales ($)', 'YearMonth': 'Month'})
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# Year over Year comparison
yearly = sales.groupby('Year').agg({
    'Sales': 'sum',
    'Gross Profit': 'sum',
    'Units': 'sum',
    'Order ID': 'nunique'
}).reset_index()

yearly['YoY Growth'] = yearly['Sales'].pct_change() * 100
yearly['Margin %'] = yearly['Gross Profit'] / yearly['Sales'] * 100

print(yearly.to_string(index=False))

In [ ]:
# Seasonality - Monthly pattern
monthly_pattern = sales.groupby('Month')['Sales'].mean()

fig, ax = plt.subplots(figsize=(10, 5))
monthly_pattern.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Average Sales by Month (Seasonality)')
ax.set_xlabel('Month')
ax.set_ylabel('Average Sales ($)')
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=0)
plt.tight_layout()
plt.show()

## 5. Product Analysis

In [ ]:
# Merge with products
sales_merged = sales.merge(products, left_on='Product ID', right_on='Product ID', how='left')

# Sales by Division
div_sales = sales_merged.groupby('Division_y').agg({
    'Sales': 'sum',
    'Gross Profit': 'sum',
    'Units': 'sum'
}).reset_index()
div_sales['Margin %'] = div_sales['Gross Profit'] / div_sales['Sales'] * 100
div_sales = div_sales.sort_values('Sales', ascending=False)

print("Sales by Division:")
print(div_sales.to_string(index=False))

In [ ]:
# Top 10 Products
top_products = sales.groupby('Product Name').agg({
    'Sales': 'sum',
    'Gross Profit': 'sum',
    'Units': 'sum'
}).sort_values('Sales', ascending=False).head(10)

fig = px.bar(top_products.reset_index(), x='Product Name', y='Sales',
             title='Top 10 Products by Sales',
             color='Gross Profit', color_continuous_scale='Greens')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 6. Geographic Analysis

In [ ]:
# Sales by State
state_sales = sales.groupby('State/Province').agg({
    'Sales': 'sum',
    'Order ID': 'nunique',
    'Customer ID': 'nunique'
}).sort_values('Sales', ascending=False).head(10)
state_sales.columns = ['Sales', 'Orders', 'Customers']

print("Top 10 States:")
print(state_sales.to_string())

In [ ]:
# Sales by Region
region_sales = sales.groupby('Region').agg({
    'Sales': 'sum',
    'Gross Profit': 'sum'
}).reset_index()

fig = px.pie(region_sales, values='Sales', names='Region',
             title='Sales Distribution by Region')
fig.show()

## 7. Customer Analysis

In [ ]:
# Customer metrics
customer_stats = sales.groupby('Customer ID').agg({
    'Sales': 'sum',
    'Order ID': 'nunique',
    'Units': 'sum'
}).reset_index()
customer_stats.columns = ['Customer ID', 'Total Sales', 'Orders', 'Units']

print("Customer Statistics:")
print(customer_stats[['Total Sales', 'Orders', 'Units']].describe())

In [ ]:
# Customer segments by sales
customer_stats['Segment'] = pd.qcut(customer_stats['Total Sales'], q=4, 
                                     labels=['Low', 'Medium', 'High', 'VIP'])

segment_summary = customer_stats.groupby('Segment').agg({
    'Customer ID': 'count',
    'Total Sales': 'sum',
    'Orders': 'mean'
}).reset_index()
segment_summary.columns = ['Segment', 'Customers', 'Total Sales', 'Avg Orders']
segment_summary['% of Sales'] = segment_summary['Total Sales'] / segment_summary['Total Sales'].sum() * 100

print("\nCustomer Segments:")
print(segment_summary.to_string(index=False))

## 8. Operational Analysis

In [ ]:
# Shipping analysis
ship_analysis = sales.groupby('Ship Mode').agg({
    'Sales': 'sum',
    'Order ID': 'nunique',
    'Ship Days': 'mean'
}).sort_values('Sales', ascending=False)
ship_analysis.columns = ['Sales', 'Orders', 'Avg Ship Days']

print("Shipping Mode Analysis:")
print(ship_analysis.to_string())

In [ ]:
# Shipping days distribution
fig, ax = plt.subplots(figsize=(10, 5))
sales['Ship Days'].hist(bins=20, ax=ax, color='coral', edgecolor='white')
ax.set_title('Distribution of Shipping Days')
ax.set_xlabel('Days to Ship')
ax.set_ylabel('Frequency')
ax.axvline(sales['Ship Days'].mean(), color='red', linestyle='--', label=f'Mean: {sales["Ship Days"].mean():.1f}')
ax.legend()
plt.tight_layout()
plt.show()

## 9. Correlation Analysis

In [ ]:
# Correlation matrix
corr_cols = ['Sales', 'Units', 'Cost', 'Gross Profit', 'Ship Days']
corr_matrix = sales[corr_cols].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=ax)
ax.set_title('Correlation Matrix')
plt.tight_layout()
plt.show()

## 10. Key Insights Summary

In [ ]:
print("="*60)
print("KEY INSIGHTS SUMMARY")
print("="*60)
print("\n📈 GROWTH:")
print(f"  - Total Revenue: ${sales['Sales'].sum():,.0f}")
print(f"  - Strong YoY growth: 2023 (+27%), 2024 (+27%)")
print(f"  - Consistent ~66% gross margin")

print("\n🍫 PRODUCTS:")
print(f"  - Chocolate dominates: 93% of revenue")
print(f"  - Top product: Wonka Bar - Triple Dazzle Caramel")
print(f"  - Chocolate has highest margin (~67%)")

print("\n📍 GEOGRAPHY:")
print(f"  - California is #1 market (20% of sales)")
print(f"  - Top 3 states = 40% of revenue")

print("\n📅 SEASONALITY:")
print(f"  - Q4 is strongest (holiday season)")
print(f"  - February is typically lowest")

print("\n🚚 OPERATIONS:")
print(f"  - Standard Class: 60% of orders")
print(f"  - Avg shipping: {sales['Ship Days'].mean():.0f} days")

print("\n💡 RECOMMENDATIONS:")
print("  1. Focus marketing on Q4 to maximize seasonal peak")
print("  2. Expand presence in California and NY")
print("  3. Consider expanding Sugar/Other divisions")
print("  4. Analyze low-demand periods (Feb) for promotions")